# Creating hardware in Python using CyriteHDL

CyriteHDL or short CyHDL is a **hardware definition toolkit** based on a kernel using an internal representation python notation to generate hardware or other code (such as another HDL or machine/micro code) procedurally. It is featured by an emulation layer to provide some (not full) language compatibility to the MyHDL dialect. It is assumed that you have some knowledge of Verilog or VHDL.

If you are aiming at a fresh start with a Python based HDL framework, this howto should get you going with the following basics:

0. Need to know [inference basics](basics.ipynb)
1. The (in)famous [blinky](blinky.ipynb)
   1. Designing a counter, state machine and configuration register
   1. Testing and simulating its function
2. Design methods and factory classes: Organizing your code to be reusable, portable...
   1. [Contexts, modules, functions](methods.ipynb)   
       * RTL functions: context sensitive functional descriptions
   2. [Simulation and verification](sim_intro.ipynb)
3. Translation or synthesis
   1. [Supported targets](targets.ipynb)
   1. [Direct synthesis to yosys](rtlil.ipynb)
5. Compiling a design for a FPGA target ( **OMITTED** )
   1. Creating the board supply package
   2. Precompiled board supply packages:
      * ECP5 Versa development kit
      * ECP5 EVDK
      * Gatemate evaluation kit (planned)

## The Cyrite reference

The Cyrite HDL is split in several layers:

* Cyrite library layer: top level HDL syntax with partial MyHDL compatibility
* myirl 'intermediate representation layer' kernel layer: Procedural, generator style syntax and library
  

When dealing with development of new extensions, one is mostly operating in the `myirl` domain. Thus, `myirl.library` elements mostly use IRL (intermediate representation layer) notation.
Some elements are only documented in the IRL domain, however can be used from the cyrite layer. This is yet not fully coverage-tested.

* [Signals, wires and operations](operations.ipynb)
* [Generator basics](generators.ipynb) - When to use `yield` and when not
* [Signal types and Interfaces](signals_interfaces.ipynb)
   * [Port classes](ports.ipynb)
   * [Tristate and undefined signals](stdlogic.ipynb)
   * [Arrays, Memories, Vectors](arrays.ipynb)
   * [Memory library](cyrite_memories.ipynb)
* [Library concepts: Organizing your code](library.ipynb)
   * [Blackboxes, external HDL modules](library.ipynb#External-HDL-libraries)
   * Inference and design modules
* Cyrite/MyIRL Library and extensions
   * [MyIRL library reference](../src/myirl/library/index.ipynb)
   * [Cross clock](xclkdomain.ipynb) domain transition auxiliaries
   * [Register concepts](registers.ipynb)
       * Bit fields
       * Partially assigneable signals
       * Address decoders, [Busses](busses.ipynb)
       * [Memory mapped range registers](../examples/soc_auxiliaries.ipynb)
    * High level synthesis: The HLS subsystem
        * [Introduction](hls.ipynb): Pipelines
        * [Vector operations](hls_vector.ipynb)
        * Arithmetics: the `flexbv` data type
        * Automated pipeline verification
* Simulation:
   * [Simulation examples](simulation.ipynb) and scenarios, waveform displays
   * [Test bench strategies](testbenches.ipynb): resuable test benches
   * [Co-Verification tricks](co-verification.ipynb) -- Simple co-process mechanisms to run Python and backend simulation in parallel (a.k.a virtual poor man's LockStep)

## Licensing and game rules

For the time being, there are no OpenSource licenses involved with the Cyrite libraries and kernel.

This distribution, i.e. the Docker container, Jupyter notebook and software setup is shared for a priori educational purposes.

There are compiled (closed source) and Python source parts in this distribution. When directly deriving from a class available in open source, you are not required to publish your derivation as Python source. You are free to distribute the (possibly obfuscated) HDL output, but not alter the copyright header. 

A few more rules apply:

* When publishing a GHDL simulation executable in particular, you are required to publish the generated VHDL code as well, as it is subject to the libgrt licensing terms by linking against it (This is a third party licensing issue).
* When you *distribute* a compiled hardware generator project for commercial purposes:
    * ..and derive from an open source class or component, you are required to publish the source as well
    * ..and derive from a closed source component, you need to contact the author via github in order to discuss licensing terms for this special case

Note that a *distribution* is considered as:
* Publishing your IP core on the web as compiled `.so` module or simulation executable
* Sending such an IP core to anyone who is not a member of your organisation